In [44]:
import os
from pyspark.sql import SparkSession
from google.colab import drive
from pyspark.sql import functions as f
import math as m
from pyspark.sql.types import DoubleType

Установка Spark

In [45]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
import findspark

Задание переменных окружения

In [46]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

Создание и настройка сессии Spark

In [47]:
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

Загрузка данных

In [48]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
stations = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Большие данные/stations.csv', header=True, inferSchema=True, timestampFormat='M/d/y')
stations.printSchema()
stations.show(5, truncate=False)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: timestamp (nullable = true)

+---+---------------------------------+------------------+-------------------+----------+--------+-------------------+
|id |name                             |lat               |long               |dock_count|city    |installation_date  |
+---+---------------------------------+------------------+-------------------+----------+--------+-------------------+
|2  |San Jose Diridon Caltrain Station|37.329732         |-121.90178200000001|27        |San Jose|2013-08-06 00:00:00|
|3  |San Jose Civic Center            |37.330698         |-121.888979        |15        |San Jose|2013-08-05 00:00:00|
|4  |Santa Clara at Almaden           |37.333988         |-121.894902        |11        |San Jose|2013-08-06 00:00:00|
|5  |Ad

In [50]:
trips = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Большие данные/trips.csv', header=True, inferSchema=True, timestampFormat='M/d/y H:m')
trips.printSchema()
trips.show(5, truncate=False)

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: timestamp (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

+----+--------+-------------------+------------------------+----------------+-------------------+------------------------+--------------+-------+-----------------+--------+
|id  |duration|start_date         |start_station_name      |start_station_id|end_date           |end_station_name        |end_station_id|bike_id|subscription_type|zip_code|
+----+--------+-------------------+------------------------+----------------+-------------------+------------------------+--------------+-------+----------------

#**Решите следующие задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):**

##**1. Найти велосипед с максимальным временем пробега**

In [51]:
# группировка по bike_id и нахождение времени пробега каждого велосипеда
grouped_by_bike_id = trips.groupBy('bike_id').sum('duration')
# нахождение максимального времени пробега
max_duration = grouped_by_bike_id.agg(f.max('sum(duration)').alias('max_duration')).first()['max_duration']
# нахождение соответствующего bike_id
bike_id = grouped_by_bike_id.filter(grouped_by_bike_id['sum(duration)'] == max_duration).first()['bike_id']

print(f'Велосипед {bike_id} имеет максимальное время пробега, равное {max_duration}')

Велосипед 535 имеет максимальное время пробега, равное 18611693


##**2. Найти наибольшее геодезическое расстояние между станциями**

In [52]:
# функция для расчёта геодезического расстояния
def get_distance(lat1, long1, lat2, long2):
  lat1, long1, lat2, long2 = m.radians(lat1), m.radians(long1), m.radians(lat2), m.radians(long2)
  # вычисляем по формуле Хаверсина
  h = 0.5 * (1 - m.cos(lat1 - lat2) + m.cos(lat1) * m.cos(lat2) * (1 - m.cos(long1 - long2)))
  return 2 * 6371 * m.asin(m.sqrt(h))

In [53]:
# создание пользовательской функции
get_distance_udf = f.udf(get_distance, DoubleType())

# получаем пары станций, исключая повторы
pairs = stations.alias("stations1").crossJoin(stations.alias("stations2")).filter(f.col("stations1.id") != f.col("stations2.id"))

# вычисление расстояния для каждой пары
distances = pairs.select(f.col("stations1.id").alias("station1"), f.col("stations2.id").alias("station2"),
  get_distance_udf(f.col("stations1.lat"), f.col("stations1.long"), f.col("stations2.lat"), f.col("stations2.long")).alias("distance"))

# поиск максимального расстояния
max_distance = distances.agg(f.max('distance').alias('max_distance')).first()['max_distance']

# для получения id станций
row = distances.filter(distances['distance'] == max_distance).first()

print(f'Наибольшее геодезическое расстояние равно {max_distance} км между станциями с id {row[0]} и {row[1]}')

Наибольшее геодезическое расстояние равно 69.9208759542579 км между станциями с id 16 и 60


##**3. Найти путь велосипеда с максимальным временем пробега через станции**

In [54]:
# поиск поездки с максимальным времененем
max_duration_trip = trips.sort("duration", ascending=False).first()
print(f'Путь велосипеда с id {max_duration_trip["bike_id"]} с максимальным времененем, равным {max_duration_trip["duration"]}, \
со станции {max_duration_trip["start_station_name"]} до станции {max_duration_trip["end_station_name"]}')

Путь велосипеда с id 535 с максимальным времененем, равным 17270400, со станции South Van Ness at Market до станции 2nd at Folsom


##**4. Найти количество велосипедов в системе**

In [55]:
# Выбираем столбец с id велосипедов, оставляем уникальные, ищем количество
count = trips.select('bike_id').distinct().count()
print(f'В системе {count} велосипедов')

В системе 700 велосипедов


##**5. Найти пользователей потративших на поездки более 3 часов**

In [56]:
# группировка по bike_id, нахождение времени пробега каждого велосипеда, фильтрация по времени пробега более 3 часов
trips.groupBy('bike_id').sum('duration').filter(f.col('sum(duration)') > 3 * 60 * 60).show()

+-------+-------------+
|bike_id|sum(duration)|
+-------+-------------+
|    471|      1718831|
|    496|      1679568|
|    148|       332138|
|    463|      1722796|
|    540|      1752835|
|    392|      1789476|
|    623|      2037219|
|    243|       307458|
|    516|      1896751|
|     31|       407907|
|    580|      1034382|
|    137|      1529200|
|    251|      1282980|
|    451|      1695574|
|     85|      1214769|
|    458|      1647080|
|     65|       216922|
|    588|       266415|
|    255|       396395|
|     53|       226389|
+-------+-------------+
only showing top 20 rows

